# Project #3 - Final: *Thinking About Fake News* from Project 2

## https://github.com/pirple/Data-Mining-With-Python/blob/master/Part%206/Section-5%20.ipynb

In [1]:
#libs and pyspark init 
import pyspark.sql.functions as F
import pyspark as ps
from pyspark import SQLContext  

spark = ps.sql.SparkSession.builder \
    .master('local[2]') \
    .appName('spark-ml') \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [2]:
true_data = 'Data/True1000.csv'
fake_data = 'Data/Fake1000.csv'

In [3]:
# read CSV Spark
df_true = sqlContext.read.csv(true_data,
                         header=True,       # use headers or not
                         quote='"',         # char for quotes
                         sep=",",           # char for separation
                         inferSchema=True)  # do we infer schema or not ?

df_fake = sqlContext.read.csv(fake_data,
                         header=True,       # use headers or not
                         quote='"',         # char for quotes
                         sep=",",           # char for separation
                         inferSchema=True)  # do we infer schema or not ?

df_true.show(5), df_fake.show(5), #df.head(2)

+--------------------+--------------------+------------+------------------+
|               title|                text|     subject|              date|
+--------------------+--------------------+------------+------------------+
|As U.S. budget fi...|WASHINGTON (Reute...|politicsNews|December 31, 2017 |
|U.S. military to ...|WASHINGTON (Reute...|politicsNews|December 29, 2017 |
|Senior U.S. Repub...|WASHINGTON (Reute...|politicsNews|December 31, 2017 |
|FBI Russia probe ...|WASHINGTON (Reute...|politicsNews|December 30, 2017 |
|Trump wants Posta...|SEATTLE/WASHINGTO...|politicsNews|December 29, 2017 |
+--------------------+--------------------+------------+------------------+
only showing top 5 rows

+--------------------+--------------------+-------+-----------------+
|               title|                text|subject|             date|
+--------------------+--------------------+-------+-----------------+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017|
| Drunk Bra

(None, None)

In [14]:
df_true.schema, df_fake.schema, df_true.dtypes #df.info()

(StructType(List(StructField(title,StringType,true),StructField(text,StringType,true),StructField(subject,StringType,true),StructField(date,StringType,true))),
 StructType(List(StructField(title,StringType,true),StructField(text,StringType,true),StructField(subject,StringType,true),StructField(date,StringType,true))),
 [('title', 'string'),
  ('text', 'string'),
  ('subject', 'string'),
  ('date', 'string')])

## I would create a Dictionary with all the words than apply to the category Fake, and for each news compare all the words, if it does not match Dictionary then belongs to the category Real, else is Fake.

# INIT python consumes API

In [42]:
import pprint
import requests     # 2.19.1
secret = 'yourAPIKEY' #API KEY
# Define the endpoint
url = 'https://newsapi.org/v2/everything?'
def getNewsApiData(phrase='big data'):
    # Specify the query and number of returns
    parameters = {
        'q': phrase, # query phrase
        'pageSize': 20,  # maximum is 100
        'apiKey': secret # your own API key
    }
    # Make the request
    response = requests.get(url, params=parameters)

    # Convert the response to JSON format and pretty print it
#     response_json = response.json()
#     pprint.pprint(response_json)
    return response.json()

In [45]:
data01 = getNewsApiData()
pprint.pprint(data01)

{'articles': [{'author': 'Brendan Hesse',
               'content': 'Of all the big tech companies, Apple has arguably '
                          'the best privacy policies. And with iOS 14, Apple '
                          'is making yet another big privacy-focused upgrade '
                          'to iOS. Once the new OS update hits this fall, us… '
                          '[+1846 chars]',
               'description': 'Of all the big tech companies, Apple has '
                              'arguably the best privacy policies. And with '
                              'iOS 14, Apple is making yet another big '
                              'privacy-focused upgrade to iOS. Once the new OS '
                              'update hits this fall, users can choose to hide '
                              'their device’s “Identifier for Advert…',
               'publishedAt': '2020-08-27T21:00:00Z',
               'source': {'id': None, 'name': 'Lifehacker.com'},
               'titl

# END python consumes API

## INIT TF IDF

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [144]:
fakeList = "Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is Disturbing"
tfidf = TfidfVectorizer()

In [149]:
def isTrue(news, show=False):
    response = tfidf.fit_transform([fakeList, news])
    feature_names = tfidf.get_feature_names()
#     print("len = ",len(response.nonzero()[1])-1)
    x = len(fakeList.split())-1
    if (show==True):
        for col in response.nonzero()[1]:
            print (feature_names[col], ' - ', response[0, col])
    tot = 0
    cnt = 0
    fake_sum = 0
    news_sum = 0
    for col in response.nonzero()[1]:
        if (cnt>x):
            news_sum += response[0, col]
        else:
            fake_sum += response[0, col]
        tot += response[0, col]
        cnt += 1
#     print("3 = ",news_sum)
#     print("2 = ",fake_sum)
#     print("1 = ",tot-fake_sum)
    print("FakeWords% = ",news_sum * 100/fake_sum)
    if (news_sum * 100/fake_sum < 70): #less than 70% of the fake words inside the news?
        return True
    else:
        return False
# isTrue(docB, False)

In [150]:
docB = "The truck is driven on the highway"
isTrue(docB)

FakeWords% =  6.075291258616697


True

In [151]:
docB = "Embarrassing New Year1’s Eve Message; This is Disturbing Donald Trump Sends Out epa ue ma eeee e dsfsdf sfsd ewafa adsf adsfafa asdf asf afwerwer wer werwefsdfsd fsdf wewe rwerwe rwer sdfg fg fgs gfs dfg"
isTrue(docB)

FakeWords% =  88.6705972258182


False

In [152]:
docB = "Embarrassing New Year1’s Eve Message; this is sdfsd sdfsdf Disturbing Donald Trump Sends Out"
isTrue(docB)

FakeWords% =  88.6705972258182


False

In [153]:
docB = "Donald Trump Sends Out this is sdfsd sdfsdf Disturbing "
isTrue(docB)

FakeWords% =  49.902598893631406


True

## END TF IDF